Testing 

In [1]:
import numpy as np
import logzero
import pickle
from pprint import pprint
from lore_explainer.datamanager import prepare_adult_dataset, prepare_dataset

from glocalx import GLocalX, shut_up_tensorflow
from rule_loaders.lore_to_glocalx import lore_to_glocalx


# Set log profile: INFO for normal logging, DEBUG for verbosity
logzero.loglevel(logzero.logging.DEBUG)
np.set_printoptions(precision=4, suppress=True)
shut_up_tensorflow()

# Load black box: optional!
# Use black_box = None to use the dataset labels
model_file = "RandomForestClassifier_for_adult"
with open(f"data/models/{model_file}.pkl", 'rb') as model_file:
    black_box = pickle.load(model_file)

In [2]:
data_filename = 'adult'
# Prepare data
if data_filename == "adult":
    df, class_name = prepare_adult_dataset(f'data/{data_filename}.csv')
else:
    raise NotImplementedError
df, feature_names, *_ = prepare_dataset(
    df, class_name)

data = df.to_numpy(dtype=np.float32)
print("Data size is: ", data.shape)

Data size is:  (32561, 104)


In [3]:
# Load local explanations
lore_rules_file = "lore_rules_adult_30"
info_file = "adult_info"
glocal_rules = lore_to_glocalx(f"data/lore_rules/{lore_rules_file}.pkl", f"data/info_files/{info_file}.json")
# pprint(glocal_rules)

In [ ]:
# Create a GLocalX instance for `black_box`
glocalx = GLocalX(model_ai=black_box, name="adult_glocal")
# Fit the model, use batch_size=128 for larger datasets
glocalx.fit(rules=glocal_rules, train_set=data, batch_size=128)

AttributeError: 'GLocalX' object has no attribute 'name'

In [15]:
glocalx.boundary

[{{3: (-inf, 44.0), 0: (-inf, 53.5), 2: (-inf, 957.5)}-> 0,
  {1: (-inf, 12111.0)}-> 0,
  {1: (-inf, 5381.0), 16: (0.5, inf)}-> 0,
  {60: (-inf, 0.5), 21: (-inf, 0.5)}-> 0},
 {{1: (-inf, 9228.0), 60: (0.5, inf)}-> 0, {3: (-inf, 39.5)}-> 0}]

In [16]:
glocalx.fine_boundary

{{3: (-inf, 44.0), 0: (-inf, 53.5), 2: (-inf, 957.5)}-> 0,
 {1: (-inf, 12111.0)}-> 0,
 {1: (-inf, 5381.0), 16: (0.5, inf)}-> 0,
 {1: (-inf, 9228.0), 60: (0.5, inf)}-> 0,
 {60: (-inf, 0.5), 21: (-inf, 0.5)}-> 0,
 {3: (-inf, 39.5)}-> 0}

In [18]:
# Retrieve global explanations by fidelity percentile
alpha = 0.5
global_explanations = glocalx.get_fine_boundary_alpha(alpha)
global_explanations

[{1: (-inf, 6155.5), 27: (0.5, inf)}-> 0,
 {3: (-inf, 48.5)}-> 0,
 {3: (47.5, 52.5), 0: (38.5, inf)}-> 1,
 {2: (1789.5, inf)}-> 1]

In [29]:
# Retrieve exactly `alpha` global explanations
alpha = 4
global_explanations = glocalx.get_fine_boundary_alpha(alpha)
global_explanations

[{}-> 0,
 {3: (-inf, 48.5)}-> 0,
 {1: (-inf, 6155.5), 27: (0.5, inf)}-> 0,
 {}-> 1,
 {3: (47.5, 52.5), 0: (38.5, inf)}-> 1,
 {2: (1789.5, inf)}-> 1]

In [26]:
glocalx.evaluator.bic(glocalx.fine_boundary)

-0.7702767789885829

In [ ]:
glocalx.evaluator.binary_fidelity_model(glocalx.fine_boundary)

In [30]:
glocalx.evaluator.perfect_coverages

{}